1. Data loading and initial exploration

Load the dataset into a pandas DataFrame
Explore the first few rows to understand the structure of data
Check the data types, summary statistics, and unique values of each column
Identify any data quality issues or inconsistencies

In [8]:
import pandas as pd

df=pd.read_csv("../CaseStudy3/SuperStore_Dataset.csv")
# Display the first 10 rows of the DataFrame    
#first_10_rows_data = df.head(10)
#print(f"First 10 rows of data:\n {first_10_rows_data} \n")
# Display the last 10 rows of the DataFrame    
#last_10_rows_data = df.tail(10)
#print(f"Last 10 rows of data: \n {last_10_rows_data}")
# Display the data types of each column in the DataFrame
#data_types_in_datasets=df.dtypes
#print(f"Data types in the dataset:\n {data_types_in_datasets} \n")
#display summary statistics of the DataFrame
#summary_statistics = df.describe()
#print(f"Summary statistics of the dataset:\n {summary_statistics} \n")
#Display unique values in all columns of the DataFrame
#unique_values = {col: df[col].unique() for col in df.columns}
#print(f"Unique values in each column:\n {unique_values} \n")
#describe the DataFrame
data_information=df.info()
print(f"Data information:\n {data_information} \n")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10014 entries, 0 to 10013
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         10014 non-null  int64  
 1   Order ID       10014 non-null  object 
 2   Order Date     10014 non-null  object 
 3   Ship Date      10014 non-null  object 
 4   Ship Mode      9916 non-null   object 
 5   Customer ID    10014 non-null  object 
 6   Customer Name  10014 non-null  object 
 7   Segment        10014 non-null  object 
 8   Country        10014 non-null  object 
 9   City           10014 non-null  object 
 10  State          10014 non-null  object 
 11  Postal Code    10014 non-null  int64  
 12  Region         10014 non-null  object 
 13  Product ID     10014 non-null  object 
 14  Category       10014 non-null  object 
 15  Sub-Category   10014 non-null  object 
 16  Product Name   10014 non-null  object 
 17  Sales Price    10014 non-null  float64
 18  Quanti